In [1]:
import os
from tqdm import tqdm
import numpy as np
import tiktoken
from datasets import load_dataset # huggingface datasets

In [12]:
dataset = load_dataset("json", data_files='./data/json/dev.json')

Found cached dataset json (/Users/bm/.cache/huggingface/datasets/json/default-14dc56ae28f0b495/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


  0%|          | 0/1 [00:00<?, ?it/s]

In [14]:
# take only first 100 examples
split_dataset = dataset['train'].train_test_split(test_size=0.1, seed=2357, shuffle=True)
split_dataset['val'] = split_dataset.pop('test') # rename the test split to val

Loading cached split indices for dataset at /Users/bm/.cache/huggingface/datasets/json/default-14dc56ae28f0b495/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4/cache-e65036e96329467f.arrow and /Users/bm/.cache/huggingface/datasets/json/default-14dc56ae28f0b495/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4/cache-507da4f4a54b4983.arrow


In [16]:
from sentence_transformers import SentenceTransformer
model_name = 'all-MiniLM-L6-v2'
model = SentenceTransformer(model_name)
# we now want to tokenize the dataset. first define the encoding function (gpt2 bpe)
def process(example):
    embds = model.encode(example['sentences'])
    out = {'id': example['id'], 'labels': example['labels'], 'embeddings': embds}
    return out

sample_dataset = split_dataset['train'].select(range(100))
# tokenize the dataset
tokenized = sample_dataset.map(
    process,
    # remove_columns=['sentences'],
    desc="tokenizing the splits",
    num_proc=1,
)

tokenizing the splits:   0%|          | 0/100 [00:00<?, ? examples/s]

In [19]:
tokenized[0]

{'sentences': ['Michael Lynn Downs (born June 9, 1959) is a former American football safety in the NFL, having played for the Dallas Cowboys (1981–1988) and the Phoenix Cardinals (1989).',
  'He is a 1977 graduate of South Oak Cliff High School in Dallas.',
  'Downs graduated from South Oak Cliff High School in Dallas, Texas.',
  'During high school, he was a member of the Honor Society.',
  'He served as team captain of the football team and was selected as an All-District in football.',
  'He also was a letterman in Track and Field.',
  'Downs graduated from Rice University in 1981.',
  'He received a BA in Business Management, Political Science and Physical Education.',
  'While in college he was selected as All-Southwest conference and All-American honorable mention in football, as well as having served as team captain.',
  'He was also punked by legendary Oilers running back, Earl Campbell.',
  'Downs was signed by the Dallas Cowboys in 1981 as an undrafted free agent safety.',
  

In [ ]:
import nltk
tokenizer = nltk.tokenize.TweetTokenizer()
text = tokenizer.tokenize('This is a cooool #dummysmiley: :-) :-P <3 and some arrows < > -> <--')
# remove stopwords using nltk
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
filtered_sentence = [w for w in text if not w in stop_words]